# 🏥 Smart Health Record Analyzer
This notebook analyzes patient records using SQLite and SQL queries.

In [1]:

import sqlite3
import pandas as pd

# Connect to SQLite database (in-memory for testing)
conn = sqlite3.connect(":memory:")
cursor = conn.cursor()

# Create Patients table
cursor.execute('''
CREATE TABLE patients (
    patient_id INTEGER PRIMARY KEY,
    name TEXT,
    age INTEGER,
    gender TEXT
);
''')

# Create Visits table (records patient visits to doctors)
cursor.execute('''
CREATE TABLE visits (
    visit_id INTEGER PRIMARY KEY,
    patient_id INTEGER,
    visit_date TEXT,
    doctor_name TEXT,
    diagnosis TEXT,
    treatment TEXT,
    FOREIGN KEY (patient_id) REFERENCES patients(patient_id)
);
''')

# Insert sample patients
patients_data = [
    (1, 'John Doe', 35, 'Male'),
    (2, 'Jane Smith', 42, 'Female'),
    (3, 'Alice Johnson', 28, 'Female'),
    (4, 'Bob Williams', 50, 'Male'),
    (5, 'Charlie Brown', 60, 'Male')
]
cursor.executemany("INSERT INTO patients VALUES (?, ?, ?, ?)", patients_data)

# Insert sample visits
visits_data = [
    (1, 1, '2024-01-10', 'Dr. Adams', 'Flu', 'Rest & hydration'),
    (2, 2, '2024-01-15', 'Dr. Lee', 'Hypertension', 'Medication'),
    (3, 3, '2024-02-05', 'Dr. Adams', 'Allergy', 'Antihistamines'),
    (4, 4, '2024-02-20', 'Dr. Brown', 'Diabetes', 'Diet control & insulin'),
    (5, 5, '2024-03-01', 'Dr. Lee', 'Flu', 'Rest & hydration'),
    (6, 1, '2024-03-10', 'Dr. Adams', 'Flu', 'Rest & hydration'),
    (7, 2, '2024-03-12', 'Dr. Lee', 'Hypertension', 'Medication'),
    (8, 3, '2024-03-15', 'Dr. Adams', 'Flu', 'Rest & hydration')
]
cursor.executemany("INSERT INTO visits VALUES (?, ?, ?, ?, ?, ?)", visits_data)

conn.commit()

print("Database setup complete!")


Database setup complete!


## 🔍 Find Most Common Diagnoses

In [2]:

query = '''
SELECT diagnosis, COUNT(*) AS count
FROM visits
GROUP BY diagnosis
ORDER BY count DESC;
'''
df_diagnoses = pd.read_sql_query(query, conn)
print(df_diagnoses)


      diagnosis  count
0           Flu      4
1  Hypertension      2
2      Diabetes      1
3       Allergy      1


## 📊 Find Monthly Disease Trends

In [3]:

query = '''
SELECT strftime('%Y-%m', visit_date) AS month, diagnosis, COUNT(*) AS cases
FROM visits
GROUP BY month, diagnosis
ORDER BY month DESC, cases DESC;
'''
df_trends = pd.read_sql_query(query, conn)
print(df_trends)


     month     diagnosis  cases
0  2024-03           Flu      3
1  2024-03  Hypertension      1
2  2024-02      Diabetes      1
3  2024-02       Allergy      1
4  2024-01  Hypertension      1
5  2024-01           Flu      1


## 👨‍⚕️ Top Doctors by Patient Count

In [4]:

query = '''
SELECT doctor_name, COUNT(*) AS patient_count
FROM visits
GROUP BY doctor_name
ORDER BY patient_count DESC;
'''
df_doctors = pd.read_sql_query(query, conn)
print(df_doctors)


  doctor_name  patient_count
0   Dr. Adams              4
1     Dr. Lee              3
2   Dr. Brown              1


## 🔁 Find Frequent Patients (More Than 1 Visit)

In [5]:

query = '''
SELECT p.name, COUNT(v.visit_id) AS visit_count
FROM patients p
JOIN visits v ON p.patient_id = v.patient_id
GROUP BY p.name
HAVING visit_count > 1
ORDER BY visit_count DESC;
'''
df_frequent_patients = pd.read_sql_query(query, conn)
print(df_frequent_patients)


            name  visit_count
0       John Doe            2
1     Jane Smith            2
2  Alice Johnson            2


## ✅ Conclusion
- This SQL-based **Health Record Analyzer** provides insights into common diseases, trends, and doctor efficiency.
- The queries can be expanded to analyze treatment effectiveness, age-based illness patterns, and more!